In [4]:
import torch
from torch.nn.utils import prune

from tqdm import tqdm

from transformers import AutoTokenizer, OPTForCausalLM, pipeline
from datasets import load_dataset

from calculate_mask import calculate_mask
from inverse_hessian import inverse_hessian
from input_prehooks import put_input_hooks
from testing_module import calculate_perp

In [8]:
#DEVICE
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# model_name = "facebook/opt-125m"
model_name = "facebook/opt-1.3b"

#Load dataset
dataset = load_dataset('c4', 'en', streaming=True)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side='left')

# Load model with pre-trained head
model = OPTForCausalLM.from_pretrained(model_name, output_attentions=True, output_hidden_states=True).to(device=device) # type: ignore

# Load generator
generator = pipeline('text-generation', model=model_name)

Downloading: 100%|██████████| 653/653 [00:00<00:00, 529kB/s]
Downloading: 100%|██████████| 899k/899k [00:00<00:00, 7.37MB/s]
Downloading: 100%|██████████| 456k/456k [00:00<00:00, 4.47MB/s]
Downloading: 100%|██████████| 441/441 [00:00<00:00, 412kB/s]
Downloading: 100%|██████████| 2.63G/2.63G [05:28<00:00, 8.01MB/s]


In [14]:
for n, m in model.named_parameters():
    if 'weight' in n and ('proj' in n or 'fc' in n):
        print(f"name is {n}, param shape is {m.shape}")

name is model.decoder.layers.0.self_attn.k_proj.weight, param shape is torch.Size([2048, 2048])
name is model.decoder.layers.0.self_attn.v_proj.weight, param shape is torch.Size([2048, 2048])
name is model.decoder.layers.0.self_attn.q_proj.weight, param shape is torch.Size([2048, 2048])
name is model.decoder.layers.0.self_attn.out_proj.weight, param shape is torch.Size([2048, 2048])
name is model.decoder.layers.0.fc1.weight, param shape is torch.Size([8192, 2048])
name is model.decoder.layers.0.fc2.weight, param shape is torch.Size([2048, 8192])
name is model.decoder.layers.1.self_attn.k_proj.weight, param shape is torch.Size([2048, 2048])
name is model.decoder.layers.1.self_attn.v_proj.weight, param shape is torch.Size([2048, 2048])
name is model.decoder.layers.1.self_attn.q_proj.weight, param shape is torch.Size([2048, 2048])
name is model.decoder.layers.1.self_attn.out_proj.weight, param shape is torch.Size([2048, 2048])
name is model.decoder.layers.1.fc1.weight, param shape is torc

In [11]:
# Calibrate model (get inputs to each layer with calibration data)

calibration_size=6
token_length=512
calibrate_on_cpu = False
calibration_batch_size=2

# First, put in forward hooks
features = {}
put_input_hooks(model=model, features=features, feature_storage_device='cpu')


# run model on batches of calibration data, then concatenate inputs
def split_model_calibration(model):
    batch_sentences = []
    for i, data in tqdm(enumerate(iter(dataset['train']))):
        if i < calibration_size + 1:
            if len(batch_sentences) >= calibration_batch_size:
                encoded_input = tokenizer(batch_sentences, return_tensors="pt", padding="max_length", max_length=token_length, truncation=True)
                model(**encoded_input, labels=encoded_input.input_ids)
                batch_sentences = []
            batch_sentences.append(data['text'])
        else:
            break

# print(torch.cuda.memory_allocated(0))
split_model_calibration(model)
# print(torch.cuda.memory_allocated(0))


model
model.decoder
model.decoder.embed_tokens
model.decoder.embed_positions
model.decoder.final_layer_norm
model.decoder.layers
model.decoder.layers.0
model.decoder.layers.0.self_attn
model.decoder.layers.0.self_attn.k_proj
model.decoder.layers.0.self_attn.v_proj
model.decoder.layers.0.self_attn.q_proj
model.decoder.layers.0.self_attn.out_proj
model.decoder.layers.0.activation_fn
model.decoder.layers.0.self_attn_layer_norm
model.decoder.layers.0.fc1
model.decoder.layers.0.fc2
model.decoder.layers.0.final_layer_norm
model.decoder.layers.1
model.decoder.layers.1.self_attn
model.decoder.layers.1.self_attn.k_proj
model.decoder.layers.1.self_attn.v_proj
model.decoder.layers.1.self_attn.q_proj
model.decoder.layers.1.self_attn.out_proj
model.decoder.layers.1.activation_fn
model.decoder.layers.1.self_attn_layer_norm
model.decoder.layers.1.fc1
model.decoder.layers.1.fc2
model.decoder.layers.1.final_layer_norm
model.decoder.layers.2
model.decoder.layers.2.self_attn
model.decoder.layers.2.self_

7it [00:38,  5.51s/it]


In [12]:
for k in features.keys():
    print(f"{k}, shape is {features[k].shape}")

model.decoder.layers.0.self_attn.in_proj, shape is torch.Size([6, 512, 2048])
model.decoder.layers.0.self_attn.out_proj, shape is torch.Size([6, 512, 2048])
model.decoder.layers.0.fc1, shape is torch.Size([3072, 2048])
model.decoder.layers.0.fc2, shape is torch.Size([3072, 8192])
model.decoder.layers.1.self_attn.in_proj, shape is torch.Size([6, 512, 2048])
model.decoder.layers.1.self_attn.out_proj, shape is torch.Size([6, 512, 2048])
model.decoder.layers.1.fc1, shape is torch.Size([3072, 2048])
model.decoder.layers.1.fc2, shape is torch.Size([3072, 8192])
model.decoder.layers.2.self_attn.in_proj, shape is torch.Size([6, 512, 2048])
model.decoder.layers.2.self_attn.out_proj, shape is torch.Size([6, 512, 2048])
model.decoder.layers.2.fc1, shape is torch.Size([3072, 2048])
model.decoder.layers.2.fc2, shape is torch.Size([3072, 8192])
model.decoder.layers.3.self_attn.in_proj, shape is torch.Size([6, 512, 2048])
model.decoder.layers.3.self_attn.out_proj, shape is torch.Size([6, 512, 2048])


In [ ]:
# print(features["model.decoder.layers.1.self_attn.out_proj"])
# print(features["model.decoder.layers.1.self_attn_layer_norm"])
print(torch.equal(features["model.decoder.layers.5.self_attn.k_proj"], features["model.decoder.layers.5.self_attn.q_proj"]))
# print(torch.equal(features["model.decoder.layers.1.final_layer_norm"], features["model.decoder.layers.1.fc1"]))
# print(features["model.decoder.layers.1.final_layer_norm"])
# print(features["model.decoder.layers.1.fc1"])
# print(features["model.decoder.layers.4.self_attn.in_proj"])
# print(features["model.decoder.layers.6.self_attn.in_proj"])

# Sparsify Model

In [ ]:
# make a dictionary to access module by name
module_lookup_dict = {}
for module_name, module_iter in model.named_modules():
    module_lookup_dict[module_name] = module_iter
EPSILON = 1e-8
SPARSENESS = .5
B = 2
Bs = 2

# function to get module name from parameter name
def get_module_name(param_name):
    if param_name[-5:] == ".bias":
        return param_name[:-5], "bias"
    elif param_name[-7:] == ".weight":
        return param_name[:-7], "weight"
    else:
        return None, None

In [ ]:
from input_prehooks import get_feature_storage_name

layer_blacklist = ['model.decoder.embed_tokens.weight', 'model.decoder.embed_tokens.bias',
'model.decoder.embed_positions.weight']

# Using calibration data (inputs to each intermediate weight layer)
# Iterate through named parameters, calculate inverse hessian and calculate mask

# without this
param_lookup_dict = {}
param_names = []
for name, param in model.named_parameters():
    param_names.append(name)
    param_lookup_dict[name] = param

with torch.no_grad():
    # for name in tqdm(param_names):
    for name in param_names:
        param = param_lookup_dict[name]

        # skip the embed layer
        if name in layer_blacklist:
            continue
        
        # skip norms which have 1 dimension
        if len(param.shape) < 2:
            continue

        module_name, param_type = get_module_name(name)

        # apply to weight layers
        if param_type == "weight":
            print(f"Doing layer {name}")
            # get layer input from features, key is get_feature_storage_name(module_name)
            # get_feature_storage_name(module_name) stores k_proj, v_proj, q_proj together
            # since they are the same input
            layer_input = features[get_feature_storage_name(module_name)].to(device=device)
            
            # calculate inverse hessian
            # check if input is flattened e.g. from 8,512,768 to 4096,768
            if len(layer_input.shape) == 2:
                inv_hess = inverse_hessian(torch.transpose(layer_input, 0, 1), epsilon=EPSILON, 
                flattened=True)

            else:
                inv_hess = inverse_hessian(torch.transpose(layer_input, 1, 2), epsilon=EPSILON,
                flattened=False)

            # calculate mask
            mask = calculate_mask(W=param, H_inv=inv_hess, p=SPARSENESS, B=B, Bs=Bs)
            
            # get module from lookup dictionary by module name
            module = module_lookup_dict[module_name]
            # apply mask
            prune.custom_from_mask(module=module, name=param_type, mask=mask)

# Save Pruned Model

In [ ]:
# SAVE PRUNED MODEL
pruned_model_name = f'opt-125m-{SPARSENESS}'
# torch.save(model,'pruned_models/' + pruned_model_name)
# model.save_pretrained(save_directory = 'pruned_models/' + pruned_model_name)

torch.save(model.state_dict(), f'pruned_models/{pruned_model_name}.pt')

In [ ]:
# LOAD SAVED MODEL

from save_pruned_model import load_into_model
import torch
from torch.nn.utils import prune
from transformers import AutoTokenizer, OPTForCausalLM, pipeline
from datasets import load_dataset
device = 'cuda' if torch.cuda.is_available() else 'cpu'

loaded_model = OPTForCausalLM.from_pretrained('facebook/opt-125m', output_attentions=True, output_hidden_states=True).to(device=device) # type: ignore

load_into_model(loaded_model, 'pruned_models/opt-125m.pt')

In [ ]:
model_name = "facebook/opt-125m"

def get_prop_zeros(model):
    return torch.sum(model.get_decoder().layers[0].self_attn.k_proj.weight == 0) / (torch.sum(model.get_decoder().layers[0].self_attn.k_proj.weight == 0) + torch.sum(model.get_decoder().layers[0].self_attn.k_proj.weight != 0))

print(get_prop_zeros(loaded_model))
print(get_prop_zeros(model))

In [23]:
def hessian(X, epsilon=0.01, flattened=False):
    X = X.double()

    if flattened:
        X_T = torch.transpose(X, 0, 1)
        identity = torch.eye(X.shape[0], dtype=torch.float64, device=device)
        # print(f"shape of x @ x_t: {torch.sum(X @ X_T, dim=0).shape}")
        H = 2 * (X @ X_T) + (epsilon * identity)
    else:
        X_T = torch.transpose(X, 1, 2)
        identity = torch.eye(X.shape[1], dtype=torch.float64, device=device)
        # print(f"shape of x @ x_t: {torch.sum(X @ X_T, dim=0).shape}")
        H = 2 * (torch.sum(X @ X_T, dim=0)) + (epsilon * identity)

    return H

test_tensor = torch.rand(2, 3, 5)
flattened_tensor = torch.flatten(test_tensor, start_dim=0, end_dim=1)
# print(inverse_hessian(torch.transpose(test_tensor, 1, 2), flattened=False))
# print(inverse_hessian(torch.transpose(flattened_tensor, 0, 1), flattened=True))

test_tensor_2 = torch.rand(2, 3, 5)
flattened_tensor_2 = torch.flatten(test_tensor_2, start_dim=0, end_dim=1)

# comb_tensor = torch.cat((test_tensor, test_tensor_2,))
comb_tensor = torch.cat((flattened_tensor, flattened_tensor_2,))
print(comb_tensor.shape)
# flattened_tensor_3 = torch.flatten(comb_tensor, start_dim=0, end_dim=1)

# print(inverse_hessian(torch.transpose(comb_tensor, 1, 2), flattened=False))
print(hessian(torch.transpose(flattened_tensor, 0, 1), flattened=True))
print(hessian(torch.transpose(flattened_tensor_2, 0, 1), flattened=True))
print(hessian(torch.transpose(comb_tensor, 0, 1), flattened=True))
print(hessian(torch.transpose(comb_tensor, 0, 1), flattened=True) - 
    (hessian(torch.transpose(flattened_tensor, 0, 1), flattened=True) + hessian(torch.transpose(flattened_tensor_2, 0, 1), flattened=True)))

    

torch.Size([12, 5])
tensor([[3.3387, 1.7186, 2.8955, 3.1412, 2.5768],
        [1.7186, 1.4465, 1.6266, 2.0901, 2.3053],
        [2.8955, 1.6266, 3.2232, 3.5412, 2.4887],
        [3.1412, 2.0901, 3.5412, 4.6508, 3.9475],
        [2.5768, 2.3053, 2.4887, 3.9475, 5.1958]], dtype=torch.float64)
tensor([[3.5618, 4.1236, 3.2586, 2.8208, 2.7146],
        [4.1236, 6.5931, 4.9056, 3.6332, 4.7593],
        [3.2586, 4.9056, 4.1975, 2.8734, 3.5105],
        [2.8208, 3.6332, 2.8734, 3.3229, 2.9970],
        [2.7146, 4.7593, 3.5105, 2.9970, 4.0479]], dtype=torch.float64)
tensor([[6.8906, 5.8422, 6.1541, 5.9621, 5.2914],
        [5.8422, 8.0296, 6.5322, 5.7233, 7.0647],
        [6.1541, 6.5322, 7.4107, 6.4146, 5.9992],
        [5.9621, 5.7233, 6.4146, 7.9637, 6.9445],
        [5.2914, 7.0647, 5.9992, 6.9445, 9.2337]], dtype=torch.float64)
False


# Sample Testing

In [ ]:
'''# REGULAR OUTPUT
dense_model = OPTForCausalLM.from_pretrained("facebook/opt-125m", output_attentions=True, output_hidden_states=True).to(device=device)
encoded_test_input = tokenizer('What did you just say to me? I will have you know', return_tensors="pt",
                                                                                    padding="max_length", 
                                                                                    max_length=token_length, 
                                                                                    truncation=True)
#print(encoded_test_input)
print('DENSE MODEL:')
with torch.no_grad():
    generated_ids = dense_model.generate(**encoded_test_input, max_new_tokens=30, num_beams=5, do_sample=True)
dense_output = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
print(f'\tOutput: {dense_output}')

print('SPARSE MODEL: ')
with torch.no_grad():
    generated_ids = model.generate(**encoded_test_input, max_new_tokens=30, num_beams=5, do_sample=True)
sparse_output = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
print(f'\tOutput: {sparse_output}')''''

In [ ]:
dense_model = OPTForCausalLM.from_pretrained("facebook/opt-125m", output_attentions=True, output_hidden_states=True).to(device=device)
encoded_test_input = tokenizer('What did you just say to me? I will have you know', return_tensors="pt",
                                                                                    padding="max_length", 
                                                                                    max_length=token_length, 
                                                                                    truncation=True)
print(torch.exp(dense_model(**encoded_test_input, labels = encoded_test_input.input_ids).loss))
print(torch.exp(model(**encoded_test_input, labels = encoded_test_input.input_ids).loss))
print(torch.exp(loaded_model(**encoded_test_input, labels = encoded_test_input.input_ids).loss))

# Perplexity Testing

In [ ]:
test_set = load_dataset('wikitext', 'wikitext-2-v1', split='test[:10%]')
tokenized_test = tokenizer(test_set['text'])

flattened_input_ids = [item for sublist in tokenized_test.input_ids for item in sublist]
flattened_input_ids = flattened_input_ids[:(len(flattened_input_ids) - (len(flattened_input_ids) % token_length))]
flattened_input_ids = torch.Tensor(flattened_input_ids).reshape(-1, token_length).int()

flattened_masks = [item for sublist in tokenized_test.attention_mask for item in sublist]
flattened_masks = flattened_masks[:(len(flattened_masks) - (len(flattened_masks) % token_length))]
flattened_masks = torch.Tensor(flattened_masks).reshape(-1, token_length).int()

test_dict = {'input_ids': flattened_input_ids, 'attention_mask': flattened_masks}

In [ ]:
model.eval()
dense_model.eval()
output = model(**test_dict, labels=test_dict['input_ids'])
output2 = dense_model(**test_dict, labels=test_dict['input_ids'])

In [ ]:
print(torch.exp(output.loss))
print(torch.exp(output2.loss))